<a href="https://colab.research.google.com/github/achett/Hierarchical-Model/blob/main/Bayesian_Hierarchical_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hierarchicalforecast
!pip install statsforecast
!pip install datasetsforecast
!pip install nixtlats>=0.1.0
!pip install darts
!pip install mlforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.2/508.2 kB 24.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.85 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.8/279.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.2/169.2 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.7 MB/s eta 0:00:00
  Created wheel for fugue-sql-antlr: filename=fugue_sql_antlr-0.2.0-py3-none-any.whl size=158196 sha256=50f3efe9e40d0e238c47ecdb09fb8a48ee7adbbe6ad78c6f8a3ecd4fac428903
  Stored in directory: /root/.cache/pip/wheels/5a/b5/4e/216953a1c711da55de29ed7ecf158b4a5bf32ef93d69ad66dd
Successfully built fugue-sql-antlr
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.4/819.4 kB 10.0 MB/s et

In [2]:
########################
# PACKAGES
########################
# !pip install -U numba statsforecast datasetsforecast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl
from datetime import datetime
import pymc as pm

# compute base forecast no coherent
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive, AutoETS, AutoCES

#obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut, MinTrace, OptimalCombination, ERM
from hierarchicalforecast.utils import aggregate

# #obtain hierarchical dataset
# from datasetsforecast.hierarchical import HierarchicalData
from nixtlats import TimeGPT


/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [132]:
inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/budgetFY23.csv'
budget = pd.read_csv(inputFile)
budget['product'].unique()

array(['MICAFUNGIN - Micafungin Sodium', 'MIRABEGRON - Mirabegron',
       'SOLIFENACN - Solifenacin Succinate',
       'TACRO_XL - Tacrolimus Extended-Release',
       'TACROLIMUS - Tacrolimus', 'TAMSUL_TAB - Tamsulosin tab',
       'ENZA - Enzalutamide', 'ENFORTUMAB - Enforumab Vedotin',
       'GILTERITNB - Gilteritinib', 'ROXADUSTNT - Roxadustant',
       'SOLIF_TAMS - Solifenacin / Tamsulosin', 'FEZO - Fezolinetant',
       'ZOLBETUXMB - Zolbetuximab', 'JOSAMYCIN - JOSAMYCIN',
       'TAMSULOSIN - Tamsulosin HCl', 'AMPHOTERCN - Amphotericin B',
       'MESALAZINE - Mesalazine', 'ESCHERICH - Escherichia coli ext',
       'REGADENOSN - Regadenoson', 'ISA_SULFAT - Isavuconazonium Sulfate',
       'NO_PCTR - Not assigned', 'ZOLPIDEM - ZOLPIDEM',
       'NITISINONE - NITISINONE', 'SITAG_IPRA - SITAG_IPRA',
       'PEFICITINI - PEFICITINI', 'QUETIAPINE - QUETIAPINE',
       'RAMOS_IRR - RAMOS_IRR', 'GABAPENTIN - GABAPENTIN',
       'LINACLOTID - LINACLOTID', 'MEPENZOLAT - MEPENZOLAT',
 

In [134]:
##############
# PARAMS
##############
fct_periods = 12
fct_st_date = '2023-04-01'
fct_end_date = '2023-12-01'

# Create hierarchical structure and constraints
hierarchy_levels = [['top_level'],
                    ['top_level', 'level2'],
                    ['top_level', 'level2', 'level3'],
                    ['top_level', 'level2', 'level3', 'bottom_level']]

inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/regional_hierarchy.csv'
r_hier = pd.read_csv(inputFile)

# Sample data
products2include = ['TAMSUL_TAB - Tamsulosin tab', 'TAMSULOSIN - Tamsulosin HCl', 'SOLIFENACN - Solifenacin Succinate', 'SOLIF_TAMS - Solifenacin / Tamsulosin', 'MIRABEGRON - Mirabegron', 'ENZA - Enzalutamide', 'ENFORTUMAB - Enforumab Vedotin','GILTERITNB - Gilteritinib']
# products2include = ['SOLIFENACN - Solifenacin Succinate', 'TAMSULOSIN - Tamsulosin HCl']


In [135]:
##############
# DATA LOAD - BUDGET
##############
inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/budgetFY23.csv'
budget = pd.read_csv(inputFile)

# Filter
budget = budget[budget['product'].isin(products2include)]

# Merge hierarchy
budget = budget.merge(r_hier, how='left', left_on='country', right_on='cost_object')


# Transform date and y
budget['ds'] = pd.to_datetime(budget['ds'])
budget['y'] = budget['y'].astype(float)

# Filter
budget = budget[budget['category']=='EQUIV_UNIT - Equivalent Units']

# Address NA values
budget['y'] = budget['y'].fillna(0)
budget['region'] = budget['region'].fillna('')
budget['country'] = budget['country'].fillna('')
budget['product'] = budget['product'].fillna('')

# Create hierarchical dataframe
budget['top_level'] = 'global'
budget.rename(columns={'region': 'level2'}, inplace=True)
budget.rename(columns={'country': 'level3'}, inplace=True)
budget.rename(columns={'product': 'bottom_level'}, inplace=True)
budget = budget[['level2', 'level3', 'bottom_level', 'top_level', 'ds', 'y']]

# Drop duplicates
budget = budget.drop_duplicates()

budget_hier_df, budget_S_df, budget_tags = aggregate(df=budget, spec=hierarchy_levels)

# Rename column y
budget_hier_df=budget_hier_df.reset_index()
budget_hier_df.rename(columns={'y': 'budget'}, inplace=True)

In [136]:
##############
# DATA LOAD - PAM
##############
inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/PAMFY23.csv'
pam = pd.read_csv(inputFile)

# Filter
pam = pam[pam['product'].isin(products2include)]

# Merge hierarchy
pam = pam.merge(r_hier, how='left', left_on='country', right_on='cost_object')

# List of date columns to transform
date_columns = ['4/1/2022', '5/1/2022', '6/1/2022', '7/1/2022', '8/1/2022',
       '9/1/2022', '10/1/2022', '11/1/2022', '12/1/2022', '1/1/2023',
       '2/1/2023', '3/1/2023', '4/1/2023', '5/1/2023', '6/1/2023', '7/1/2023',
       '8/1/2023', '9/1/2023', '10/1/2023', '11/1/2023', '12/1/2023',
       '1/1/2024', '2/1/2024', '3/1/2024', '4/1/2024', '5/1/2024', '6/1/2024',
       '7/1/2024', '8/1/2024', '9/1/2024', '10/1/2024', '11/1/2024',
       '12/1/2024', '1/1/2025', '2/1/2025', '3/1/2025']

# Melting the DataFrame
pam = pam.melt(id_vars=['region','country', 'product', 'Version', 'Category'], value_vars=date_columns, var_name='ds', value_name='y')

# Transform date
pam['ds'] = pd.to_datetime(pam['ds'])

# Filter and merge
pam = pam[pam['Category']=='Volume']
# pam['pam'] = pam['pam'].astype(float)*1000000

# Address NA values
pam['y'] = pam['y'].fillna(0)
pam['region'] = pam['region'].fillna('')
pam['product'] = pam['product'].fillna('')

# Create hierarchical dataframe
pam['top_level'] = 'global'
pam.rename(columns={'region': 'level2'}, inplace=True)
pam.rename(columns={'country': 'level3'}, inplace=True)
pam.rename(columns={'product': 'bottom_level'}, inplace=True)
pam = pam[['level2', 'level3', 'bottom_level', 'top_level', 'ds', 'y']]

# Drop duplicates
pam = pam.drop_duplicates()

pam_hier_df, pam_S_df, pam_tags = aggregate(df=pam, spec=hierarchy_levels)

# Rename column y
pam_hier_df=pam_hier_df.reset_index()
pam_hier_df.rename(columns={'y': 'pam'}, inplace=True)

pam_hier_df = pam_hier_df.drop_duplicates()

In [137]:
########################
# DATA LOAD - ACTUALS
########################
inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/volume_output.csv'
data_raw = pd.read_csv(inputFile)

# Filter
data = data_raw[data_raw['product'].isin(products2include)]

# Merge hierarchy
data = data.merge(r_hier, how='left', on='cost_object')

# Update columns
data['ds'] = pd.to_datetime(data['ds'])
data.rename(columns={'value': 'y'}, inplace=True)

# Address NA values
data['y'] = data['y'].fillna(0)
data['region'] = data['region'].fillna('')
data['product'] = data['product'].fillna('')

# Create hierarchical dataframe
data['top_level'] = 'global'
data.rename(columns={'region': 'level2'}, inplace=True)
data.rename(columns={'cost_object': 'level3'}, inplace=True)
data.rename(columns={'product': 'bottom_level'}, inplace=True)
data = data[['level2', 'level3', 'bottom_level', 'top_level', 'ds', 'y']]

Y_hier_df, S_df, tags = aggregate(df=data, spec=hierarchy_levels)
Y_hier_df = Y_hier_df.reset_index()

#split train/test sets
Y_test_df  = Y_hier_df.groupby('unique_id').tail(fct_periods)
Y_train_df = Y_hier_df.drop(Y_test_df.index)

In [108]:
########################
# DATA LOAD - PRICE
########################
inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/price_output.csv'
data_p = pd.read_csv(inputFile)

# Filter
data_p = data_p[data_p['product'].isin(products2include)]

# Merge hierarchy
data_p = data_p.merge(r_hier, how='left', on='cost_object')

# Update columns
data_p['ds'] = pd.to_datetime(data_p['ds'])
data_p.rename(columns={'Smoothed_Price': 'y'}, inplace=True)

# Address NA values
data_p['y'] = data_p['y'].fillna(0)
data_p.replace([np.inf, -np.inf], 0, inplace=True)
data_p['region'] = data_p['region'].fillna('')
data_p['product'] = data_p['product'].fillna('')


# Create hierarchical dataframe
data_p['top_level'] = 'global'
data_p.rename(columns={'region': 'level2'}, inplace=True)
data_p.rename(columns={'cost_object': 'level3'}, inplace=True)
data_p.rename(columns={'product': 'bottom_level'}, inplace=True)
data_p = data_p[['level2', 'level3', 'bottom_level', 'top_level', 'ds', 'y']]

Y_hier_df_p, S_df_cov, tags_cov = aggregate(df=data_p, spec=hierarchy_levels)
Y_hier_df_p = Y_hier_df_p.reset_index()

In [109]:
########################
# DATA LOAD - A&P + FIELD SALES
########################
inputFile = '/content/drive/MyDrive/Colab Notebooks/SGA Prediction/data/sga_output.csv'
data_sga = pd.read_csv(inputFile)

# Filter
data_sga2 = data_sga[data_sga['product'].isin(products2include)]

# Merge hierarchy
data_sga2 = data_sga2.merge(r_hier, how='left', on='cost_object')

# Update ds
data_sga2['ds'] = pd.to_datetime(data_sga2['ds'])

# Address NA values
data_sga2.replace([np.inf, -np.inf], 0, inplace=True)
data_sga2['region'] = data_sga2['region'].fillna('')
data_sga2['product'] = data_sga2['product'].fillna('')

# Create hierarchical dataframe
data_sga2['top_level'] = 'global'
data_sga2.rename(columns={'region': 'level2'}, inplace=True)
data_sga2.rename(columns={'cost_object': 'level3'}, inplace=True)
data_sga2.rename(columns={'product': 'bottom_level'}, inplace=True)
data_sga2 = data_sga2[['level2', 'level3', 'bottom_level', 'top_level', 'ds', 'Smoothed_AP', 'Smoothed_Field_Sales']]

# Break out AP and Field_sales
data_ap = data_sga2[['level2', 'level3', 'bottom_level', 'top_level', 'ds', 'Smoothed_AP']]
data_fs = data_sga2[['level2', 'level3', 'bottom_level', 'top_level', 'ds', 'Smoothed_Field_Sales']]

data_ap.rename(columns={'Smoothed_AP': 'y'}, inplace=True)
data_fs.rename(columns={'Smoothed_Field_Sales': 'y'}, inplace=True)

data_ap['y'] = data_ap['y'].fillna(0)
data_fs['y'] = data_fs['y'].fillna(0)

<ipython-input-109-3c49cf192d77>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ap.rename(columns={'Smoothed_AP': 'y'}, inplace=True)
<ipython-input-109-3c49cf192d77>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fs.rename(columns={'Smoothed_Field_Sales': 'y'}, inplace=True)
<ipython-input-109-3c49cf192d77>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_a

In [9]:
########################
# CREATE CAUSAL DATAFRAME
########################
data_ap.rename(columns={'y': 'AP'}, inplace=True)
data_fs.rename(columns={'y': 'FieldSales'}, inplace=True)
data_p.rename(columns={'y': 'Price'}, inplace=True)

data_causal = data.merge(data_ap, on=['level2', 'level3', 'bottom_level', 'top_level', 'ds'], how='left')
data_causal = data_causal.merge(data_fs, on=['level2', 'level3', 'bottom_level', 'top_level', 'ds'], how='left')
data_causal = data_causal.merge(data_p, on=['level2', 'level3', 'bottom_level', 'top_level', 'ds'], how='left')

data_causal.fillna(0, inplace=True)

data_causal['unique_id'] = data_causal['top_level'] + '/' + data_causal['level2'] + '/' + data_causal['level3'] + '/' + data_causal['bottom_level']

data_causal.head()

<ipython-input-9-cf573fa2974e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ap.rename(columns={'y': 'AP'}, inplace=True)
<ipython-input-9-cf573fa2974e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fs.rename(columns={'y': 'FieldSales'}, inplace=True)


,level2,level3,bottom_level,top_level,ds,y,AP,FieldSales,Price,unique_id
0,Greater China,D_CN_TOTAL - China Total,SOLIFENACN - Solifenacin Succinate,global,2014-04-01,0.0,0.0,0.0,0.0,global/Greater China/D_CN_TOTAL - China Total/...
1,Greater China,D_CN_TOTAL - China Total,SOLIFENACN - Solifenacin Succinate,global,2014-05-01,0.0,0.0,0.0,0.0,global/Greater China/D_CN_TOTAL - China Total/...
2,Greater China,D_CN_TOTAL - China Total,SOLIFENACN - Solifenacin Succinate,global,2014-06-01,0.0,0.0,0.0,0.0,global/Greater China/D_CN_TOTAL - China Total/...
3,Greater China,D_CN_TOTAL - China Total,SOLIFENACN - Solifenacin Succinate,global,2014-07-01,0.0,0.0,0.0,0.0,global/Greater China/D_CN_TOTAL - China Total/...
4,Greater China,D_CN_TOTAL - China Total,SOLIFENACN - Solifenacin Succinate,global,2014-08-01,0.0,0.0,0.0,0.0,global/Greater China/D_CN_TOTAL - China Total/...


In [10]:
########################
# GRANGER CAUSALITY
########################
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests

# Assuming data_causal is your DataFrame
max_lag = 10  # Define the maximum number of lags you want to test
significance_level = 0.2  # Define your significance level

# Function to test Granger Causality and find the optimal number of lags
def find_optimal_lags(df, variable, max_lag, significance_level):
    optimal_lags = {}
    for unique_id, group in df.groupby(['level2', 'level3', 'bottom_level', 'top_level']):
        # Check if there are at least two different non-zero values in the entire series for both 'y' and the variable
        if (len(set(group[variable]) - {0}) <= 1) or (len(set(group['y']) - {0}) <= 1):
            print(f"Skipping ID {unique_id} due to insufficient non-zero variation in {variable} or y")
            optimal_lags[unique_id] = None  # Mark as None since Granger causality cannot be tested properly
            continue  # Skip this group

        print(f"Testing for ID: {unique_id} with variable {variable}")
        min_p_value = 1.0  # Initialize the minimum p-value
        optimal_lag = 0
        for lag in range(1, max_lag + 1):
            try:
                # Perform Granger causality test for specified variable
                test_result = grangercausalitytests(group[['y', variable]], maxlag=lag)
                p_value = test_result[lag][0]['ssr_chi2test'][1]  # Get the p-value
                if p_value < min_p_value:
                    min_p_value = p_value
                    optimal_lag = lag
            except ValueError:  # Skip if there are other data issues
                continue
        # Check if the minimum p-value found is below the significance level
        if min_p_value < significance_level:
            optimal_lags[unique_id] = optimal_lag
        else:
            optimal_lags[unique_id] = None  # Mark as None if no significant causality found

    # Convert dictionary to DataFrame
    optimal_lags_df = pd.DataFrame(list(optimal_lags.items()), columns=['UniqueID', 'OptimalLag'])
    optimal_lags_df[['level2', 'level3', 'bottom_level', 'top_level']] = pd.DataFrame(optimal_lags_df['UniqueID'].tolist(), index=optimal_lags_df.index)
    optimal_lags_df.drop('UniqueID', axis=1, inplace=True)

    return optimal_lags_df


# Apply the function to your dataset
optimal_lags_ap = find_optimal_lags(data_causal, 'AP', max_lag, significance_level)
optimal_lags_fs = find_optimal_lags(data_causal, 'FieldSales', max_lag, significance_level)
optimal_lags_p = find_optimal_lags(data_causal, 'Price', max_lag, significance_level)

# Create ID
optimal_lags_ap['unique_id'] = optimal_lags_ap['top_level'] + '/' + optimal_lags_ap['level2'] + '/' + optimal_lags_ap['level3'] + '/' + optimal_lags_ap['bottom_level']
optimal_lags_fs['unique_id'] = optimal_lags_fs['top_level'] + '/' + optimal_lags_fs['level2'] + '/' + optimal_lags_fs['level3'] + '/' + optimal_lags_fs['bottom_level']
optimal_lags_p['unique_id'] = optimal_lags_p['top_level'] + '/' + optimal_lags_p['level2'] + '/' + optimal_lags_p['level3'] + '/' + optimal_lags_p['bottom_level']

Streaming output truncated to the last 5000 lines.
parameter F test:         F=0.8359  , p=0.4770  , df_denom=107, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.5820  , p=0.6763  , df_denom=104, df_num=4
ssr based chi2 test:   chi2=2.5296  , p=0.6393  , df=4
likelihood ratio test: chi2=2.5017  , p=0.6443  , df=4
parameter F test:         F=0.5820  , p=0.6763  , df_denom=104, df_num=4

Granger Causality
number of lags (no zero) 5
ssr based F test:         F=0.5307  , p=0.7526  , df_denom=101, df_num=5
ssr based chi2 test:   chi2=2.9424  , p=0.7089  , df=5
likelihood ratio test: chi2=2.9045  , p=0.7147  , df=5
parameter F test:         F=0.5307  , p=0.7526  , df_denom=101, df_num=5

Granger Causality
number of lags (no zero) 6
ssr based F test:         F=0.4719  , p=0.8277  , df_denom=98, df_num=6
ssr based chi2 test:   chi2=3.2071  , p=0.7824  , df=6
likelihood ratio test: chi2=3.1616  , p=0.7883  , df=6
parameter F test:         F=0.4719  , p=0.82

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 4, but rank is 3
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 4, but rank is 3
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 5, but rank is 3
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 4, but rank is 3
  warnings.warn('covariance of constraints does not have full '
/usr/local/l

likelihood ratio test: chi2=0.0727  , p=1.0000  , df=6
parameter F test:         F=0.0221  , p=0.9955  , df_denom=101, df_num=3

Granger Causality
number of lags (no zero) 7
ssr based F test:         F=0.0103  , p=1.0000  , df_denom=99, df_num=7
ssr based chi2 test:   chi2=0.0798  , p=1.0000  , df=7
likelihood ratio test: chi2=0.0798  , p=1.0000  , df=7
parameter F test:         F=0.0239  , p=0.9950  , df_denom=99, df_num=3

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0427  , p=0.8366  , df_denom=113, df_num=1
ssr based chi2 test:   chi2=0.0439  , p=0.8341  , df=1
likelihood ratio test: chi2=0.0439  , p=0.8341  , df=1
parameter F test:         F=0.0427  , p=0.8366  , df_denom=113, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.0427  , p=0.9582  , df_denom=110, df_num=2
ssr based chi2 test:   chi2=0.0893  , p=0.9563  , df=2
likelihood ratio test: chi2=0.0893  , p=0.9563  , df=2
parameter F test:         F=0.0427  , p=0

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 5, but rank is 3
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 3
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 7, but rank is 3
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 8, but rank is 3
  warnings.warn('covariance of constraints does not have full '
/usr/local/l



Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.0486  , p=0.9526  , df_denom=110, df_num=2
ssr based chi2 test:   chi2=0.1016  , p=0.9505  , df=2
likelihood ratio test: chi2=0.1015  , p=0.9505  , df=2
parameter F test:         F=0.0486  , p=0.9526  , df_denom=110, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0563  , p=0.8128  , df_denom=113, df_num=1
ssr based chi2 test:   chi2=0.0578  , p=0.8099  , df=1
likelihood ratio test: chi2=0.0578  , p=0.8100  , df=1
parameter F test:         F=0.0563  , p=0.8128  , df_denom=113, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.0486  , p=0.9526  , df_denom=110, df_num=2
ssr based chi2 test:   chi2=0.1016  , p=0.9505  , df=2
likelihood ratio test: chi2=0.1015  , p=0.9505  , df=2
parameter F test:         F=0.0486  , p=0.9526  , df_denom=110, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.0340  , p=0.9915 

...

Streaming output truncated to the last 5000 lines.
likelihood ratio test: chi2=8.4756  , p=0.0144  , df=2
parameter F test:         F=4.2067  , p=0.0174  , df_denom=110, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.2019  , p=0.3127  , df_denom=107, df_num=3
ssr based chi2 test:   chi2=3.8417  , p=0.2791  , df=3
likelihood ratio test: chi2=3.7784  , p=0.2864  , df=3
parameter F test:         F=1.2019  , p=0.3127  , df_denom=107, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.5825  , p=0.6760  , df_denom=104, df_num=4
ssr based chi2 test:   chi2=2.5317  , p=0.6390  , df=4
likelihood ratio test: chi2=2.5037  , p=0.6440  , df=4
parameter F test:         F=0.5825  , p=0.6760  , df_denom=104, df_num=4

Granger Causality
number of lags (no zero) 5
ssr based F test:         F=0.6215  , p=0.6837  , df_denom=101, df_num=5
ssr based chi2 test:   chi2=3.4458  , p=0.6316  , df=5
likelihood ratio test: chi2=3.3938  , p=0.6

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 3
  warnings.warn('covariance of constraints does not have full '


parameter F test:         F=22.5647 , p=0.0000  , df_denom=107, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=28.9726 , p=0.0000  , df_denom=104, df_num=4
ssr based chi2 test:   chi2=125.9193, p=0.0000  , df=4
likelihood ratio test: chi2=84.6074 , p=0.0000  , df=4
parameter F test:         F=28.9726 , p=0.0000  , df_denom=104, df_num=4

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=71.0685 , p=0.0000  , df_denom=113, df_num=1
ssr based chi2 test:   chi2=72.9552 , p=0.0000  , df=1
likelihood ratio test: chi2=56.5987 , p=0.0000  , df=1
parameter F test:         F=71.0685 , p=0.0000  , df_denom=113, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=53.9650 , p=0.0000  , df_denom=110, df_num=2
ssr based chi2 test:   chi2=112.8359, p=0.0000  , df=2
likelihood ratio test: chi2=78.6247 , p=0.0000  , df=2
parameter F test:         F=53.9650 , p=0.0000  , df_denom=110, df_num=2

Granger Causality
nu

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 9, but rank is 5
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 9, but rank is 5
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 3
  warnings.warn('covariance of constraints does not have full '


Streaming output truncated to the last 5000 lines.
ssr based chi2 test:   chi2=4.2847  , p=0.1174  , df=2
likelihood ratio test: chi2=4.2068  , p=0.1220  , df=2
parameter F test:         F=2.0492  , p=0.1337  , df_denom=110, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=8.4237  , p=0.0045  , df_denom=113, df_num=1
ssr based chi2 test:   chi2=8.6473  , p=0.0033  , df=1
likelihood ratio test: chi2=8.3402  , p=0.0039  , df=1
parameter F test:         F=8.4237  , p=0.0045  , df_denom=113, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.0492  , p=0.1337  , df_denom=110, df_num=2
ssr based chi2 test:   chi2=4.2847  , p=0.1174  , df=2
likelihood ratio test: chi2=4.2068  , p=0.1220  , df=2
parameter F test:         F=2.0492  , p=0.1337  , df_denom=110, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.7708  , p=0.5128  , df_denom=107, df_num=3
ssr based chi2 test:   chi2=2.4636  , p=0.4

In [12]:
########################
# CREATE LAGGED DF
########################
optimal_lags_p.rename(columns={'OptimalLag': 'OptimalLag_p'}, inplace=True)
optimal_lags_ap.rename(columns={'OptimalLag': 'OptimalLag_ap'}, inplace=True)
optimal_lags_fs.rename(columns={'OptimalLag': 'OptimalLag_fs'}, inplace=True)

# Merge
merged_df = pd.merge(data_causal, optimal_lags_p, on=['unique_id', 'level2', 'level3', 'bottom_level', 'top_level'])
merged_df = pd.merge(merged_df, optimal_lags_ap, on=['unique_id', 'level2', 'level3', 'bottom_level', 'top_level'])
merged_df = pd.merge(merged_df, optimal_lags_fs, on=['unique_id', 'level2', 'level3', 'bottom_level', 'top_level'])

# Function to apply the lag based on 'OptimalLag', set to NaN if 'OptimalLag' is NaN
def apply_optimal_lags(group, old_column, new_column, lag_column):
    # Check if the 'OptimalLag' is NaN
    if pd.isna(group[lag_column].iloc[0]):
        group[new_column] = 0  # Set the entire column to NaN
    else:
        optimal_lag = int(group[lag_column].iloc[0])  # Convert OptimalLag to integer, assuming it's float
        # Shift the old_column and replace NaNs with 0
        group[new_column] = group[old_column].shift(optimal_lag).fillna(0)
    return group


# Applying the function to each group
lagged_data = merged_df.groupby('unique_id').apply(lambda group: apply_optimal_lags(group, 'Price', 'Price_l', 'OptimalLag_p'))
lagged_data = lagged_data.groupby('unique_id').apply(lambda group: apply_optimal_lags(group, 'AP', 'AP_l', 'OptimalLag_ap'))
lagged_data = lagged_data.groupby('unique_id').apply(lambda group: apply_optimal_lags(group, 'FieldSales', 'FieldSales_l', 'OptimalLag_fs'))


<ipython-input-12-e8aa2214a660>:26: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  lagged_data = merged_df.groupby('unique_id').apply(lambda group: apply_optimal_lags(group, 'Price', 'Price_l', 'OptimalLag_p'))
<ipython-input-12-e8aa2214a660>:27: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  lagg

In [13]:
########################
# COVARIATE MODEL
########################

from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
import os

os.environ['NIXTLA_ID_AS_COL'] = '1'


# Create X and Y
Y_ts = data_causal[['unique_id', 'ds', 'y']]
X_ts = lagged_data[['unique_id', 'ds', 'Price_l', 'AP_l', 'FieldSales_l']]
X_ts['unique_id'] = X_ts.unique_id.astype(str)

# Extract dates for train and test set
dates = Y_ts['ds'].unique()
dtrain = dates[:-fct_periods]
dtest = dates[-fct_periods:]

# Create train and test set
Y_train = Y_ts.query('ds in @dtrain')
Y_test = Y_ts.query('ds in @dtest')

X_train = X_ts.query('ds in @dtrain')
X_test = X_ts.query('ds in @dtest')

# Add exogenous
train = Y_train.merge(X_ts, how = 'left', on = ['unique_id', 'ds'])

# Create a list with the model and its instantiation parameters
models = [AutoARIMA(season_length=12)]

sf = StatsForecast(
    models=models,
    freq='MS',
    n_jobs=-1
)

def forecast_for_group(uids, train_df, X_test_df, columns, sf_model, fct_periods):
    # Filter based on unique_ids
    train_filtered = train_df[train_df['unique_id'].isin(uids)]
    X_test_filtered = X_test_df[X_test_df['unique_id'].isin(uids)]

    if columns:
        train_filtered = train_filtered[['unique_id', 'ds', 'y'] + columns]
        X_test_filtered = X_test_filtered[['unique_id', 'ds'] + columns]
        # Forecasting with exogenous variables
        fcst_gc = sf_model.forecast(df=train_filtered, h=fct_periods, X_df=X_test_filtered, level=[95])

    else:
        train_filtered = train_filtered[['unique_id', 'ds', 'y']]
        X_test_filtered = None if X_test_filtered.empty else X_test_filtered[['unique_id', 'ds']]
        # Forecasting without exogenous variables
        fcst_gc = sf_model.forecast(df=train_filtered, h=fct_periods, level=[95])

    return fcst_gc


aggregated_df = X_train.groupby('unique_id').agg({
    'Price_l': ['max', 'min'],
    'AP_l': ['max', 'min'],
    'FieldSales_l': ['max', 'min']
}).reset_index()

# Flatten MultiIndex columns
aggregated_df.columns = ['_'.join(col).strip() if col[1] else col[0] for col in aggregated_df.columns.values]

# Calculate diff
aggregated_df['price_diff'] = aggregated_df['Price_l_max'] - aggregated_df['Price_l_min']
aggregated_df['ap_diff'] = aggregated_df['AP_l_max'] - aggregated_df['AP_l_min']
aggregated_df['fieldsales_diff'] = aggregated_df['FieldSales_l_max'] - aggregated_df['FieldSales_l_min']

# Address negatives
aggregated_df.loc[(aggregated_df['Price_l_max'] < 0) | (aggregated_df['Price_l_min'] < 0), 'price_diff'] = 0
aggregated_df.loc[(aggregated_df['AP_l_max'] < 0) | (aggregated_df['AP_l_min'] < 0), 'ap_diff'] = 0
aggregated_df.loc[(aggregated_df['FieldSales_l_max'] < 0) | (aggregated_df['FieldSales_l_min'] < 0), 'fieldsales_diff'] = 0

uid_paf = aggregated_df[(aggregated_df['price_diff'] != 0) & (aggregated_df['ap_diff'] != 0) & (aggregated_df['fieldsales_diff'] != 0)]['unique_id'].values
uid_af = aggregated_df[(aggregated_df['price_diff'] == 0) & (aggregated_df['ap_diff'] != 0) & (aggregated_df['fieldsales_diff'] != 0)]['unique_id'].values
uid_pf = aggregated_df[(aggregated_df['price_diff'] != 0) & (aggregated_df['ap_diff'] == 0) & (aggregated_df['fieldsales_diff'] != 0)]['unique_id'].values
uid_pa = aggregated_df[(aggregated_df['price_diff'] != 0) & (aggregated_df['ap_diff'] != 0) & (aggregated_df['fieldsales_diff'] == 0)]['unique_id'].values
uid_f = aggregated_df[(aggregated_df['price_diff'] == 0) & (aggregated_df['ap_diff'] == 0) & (aggregated_df['fieldsales_diff'] != 0)]['unique_id'].values
uid_a = aggregated_df[(aggregated_df['price_diff'] == 0) & (aggregated_df['ap_diff'] != 0) & (aggregated_df['fieldsales_diff'] == 0)]['unique_id'].values
uid_p = aggregated_df[(aggregated_df['price_diff'] != 0) & (aggregated_df['ap_diff'] == 0) & (aggregated_df['fieldsales_diff'] == 0)]['unique_id'].values
uid = aggregated_df[(aggregated_df['price_diff'] == 0) & (aggregated_df['ap_diff'] == 0) & (aggregated_df['fieldsales_diff'] == 0)]['unique_id'].values

# Use the new function to generate forecasts
fcst_paf = forecast_for_group(uids=uid_paf, train_df=train, X_test_df=X_test, columns=['Price_l', 'AP_l', 'FieldSales_l'], sf_model=sf, fct_periods=fct_periods)
fcst_af = forecast_for_group(uids=uid_af, train_df=train, X_test_df=X_test, columns=['AP_l', 'FieldSales_l'], sf_model=sf, fct_periods=fct_periods)
fcst_pf = forecast_for_group(uids=uid_pf, train_df=train, X_test_df=X_test, columns=['Price_l', 'FieldSales_l'], sf_model=sf, fct_periods=fct_periods)
fcst_pa = forecast_for_group(uids=uid_paf, train_df=train, X_test_df=X_test, columns=['Price_l', 'AP_l'], sf_model=sf, fct_periods=fct_periods)
fcst_f = forecast_for_group(uids=uid_f, train_df=train, X_test_df=X_test, columns=['FieldSales_l'], sf_model=sf, fct_periods=fct_periods)
fcst_a = forecast_for_group(uids=uid_a, train_df=train, X_test_df=X_test, columns=['AP_l'], sf_model=sf, fct_periods=fct_periods)
fcst_p = forecast_for_group(uids=uid_p, train_df=train, X_test_df=X_test, columns=[], sf_model=sf, fct_periods=fct_periods)
fcst_x = forecast_for_group(uids=uid, train_df=train, X_test_df=X_test, columns=[], sf_model=sf, fct_periods=fct_periods)

fcst_gc = pd.concat([fcst_paf, fcst_af, fcst_pf, fcst_pa, fcst_f, fcst_a, fcst_p, fcst_x])

<ipython-input-13-d1401f26c731>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_ts['unique_id'] = X_ts.unique_id.astype(str)


In [79]:
########################
# RUN TIMEGPT + PRICE + AP + FIELD SALES
########################
#split train/test sets
Y_test_df_fap  = lagged_data.groupby('unique_id').tail(fct_periods)
Y_train_df_fap = lagged_data.drop(Y_test_df_fap.index)

# Create covariate dataframes
Y_train_df_f = Y_train_df_fap[['unique_id', 'ds', 'FieldSales_l']]
Y_train_df_a = Y_train_df_fap[['unique_id', 'ds', 'AP_l']]
Y_train_df_p = Y_train_df_fap[['unique_id', 'ds', 'Price_l']]

Y_train_df_f = Y_train_df_f.rename(columns={'FieldSales_l': 'y'})
Y_train_df_a = Y_train_df_a.rename(columns={'AP_l': 'y'})
Y_train_df_p = Y_train_df_p.rename(columns={'Price_l': 'y'})

# Create Model
model_cov = StatsForecast(
    models=[AutoETS(season_length=12)],
    freq='MS',n_jobs=-1)

# Forecast
Y_hat_df_f = model_cov.forecast(df=Y_train_df_f, h=fct_periods)

# Rename
Y_hat_df_f.rename(columns={'AutoETS': 'FieldSales_l'}, inplace=True)
Y_train_df_f.rename(columns={'y': 'FieldSales_l'}, inplace=True)

# Prep dependent variable and and covariates
# Y_train_df = Y_train_df_fap[['unique_id', 'ds', 'y', 'FieldSales_l', 'AP_l', 'Price_l']]
Y_train_df = Y_train_df_fap[['unique_id', 'ds', 'y', 'FieldSales_l']]

# Run TimeGPT
timegpt = TimeGPT(
    # defaults to os.environ.get("TIMEGPT_TOKEN")
    token = 'CgM6BBwvTfRrFetlmNCxZzRUAmSRCXndbU1CQJhPryVhgxWp0WgIDIpV9xagvCbg4ZH3TqG012wKMGNs8L7voLeNiydi5U6nFZtnu1P3VJrqECHKzZxCyef8FwNroHEkHM9vR5ltTGAFYW7bOgiiBRzjrdzgk2FIJFsLpXAP9GgMGqa0dVd8it3zbWQ9d02T7JBf9ikAPZyy1bIU206eQZV1Zdd1Rsd9a0PMjUyWxlgg0mckf59d7lwDhzGVBUpd'
)

# Forecast
tgpt_cov = timegpt.forecast(df=Y_train_df, X_df=Y_hat_df_f, h=fct_periods, freq='MS', time_col='ds', target_col='y')
tgpt_cov.head()

In [138]:
########################
# RUN HIERARCHICAL
########################
#split train/test sets
Y_test_df  = Y_hier_df.groupby('unique_id').tail(fct_periods)
Y_train_df = Y_hier_df.drop(Y_test_df.index)

# Compute base predictions
fcst = StatsForecast(df=Y_train_df,
                     models=[AutoETS(season_length=12)],
                     freq='MS', n_jobs=-1)

Y_hat_df = fcst.forecast(h=fct_periods)

# Reconcile the base predictions
reconcilers = [
    TopDown(method='forecast_proportions')
    # OptimalCombination(method = 'ols', nonnegative=True)
    # BottomUp()
    # ERM(method='closed')
]

hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_train_df,
                          S=S_df, tags=tags)

# Reset columns
Y_rec_df = Y_rec_df[['ds', 'AutoETS','AutoETS/TopDown_method-forecast_proportions']]

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:399: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:399: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/hierarchicalforecast/methods.py:286: RuntimeWarning: invalid value encountered in divide
  reconciled[idx_child] = y_hat[idx_child] * fcst_parent / childs_sum


In [88]:
########################
# RUN TIMEGPT
########################
#split train/test sets
Y_test_df  = Y_hier_df.groupby('unique_id').tail(fct_periods)
Y_train_df = Y_hier_df.drop(Y_test_df.index)

timegpt = TimeGPT(
    # defaults to os.environ.get("TIMEGPT_TOKEN")
    token = 'CgM6BBwvTfRrFetlmNCxZzRUAmSRCXndbU1CQJhPryVhgxWp0WgIDIpV9xagvCbg4ZH3TqG012wKMGNs8L7voLeNiydi5U6nFZtnu1P3VJrqECHKzZxCyef8FwNroHEkHM9vR5ltTGAFYW7bOgiiBRzjrdzgk2FIJFsLpXAP9GgMGqa0dVd8it3zbWQ9d02T7JBf9ikAPZyy1bIU206eQZV1Zdd1Rsd9a0PMjUyWxlgg0mckf59d7lwDhzGVBUpd'
)
tgpt = timegpt.forecast(Y_train_df, h=fct_periods, freq='MS', time_col='ds', target_col='y', level=[80, 90])

# Create hierarchical forecast
tgpt_h = tgpt.set_index('unique_id')
tgpt_h = tgpt_h[['ds', 'TimeGPT']]

# Reconcile the base predictions
reconcilers = [
    TopDown(method='forecast_proportions')
    # OptimalCombination(method = 'ols', nonnegative=True)
    # BottomUp()
    # ERM(method='closed')
]

hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df_tgpt = hrec.reconcile(Y_hat_df=tgpt_h, Y_df=Y_train_df,
                          S=S_df, tags=tags)

In [ ]:
# ########################
# # RUN LGBM
# ########################
# import lightgbm as lgb
# import xgboost as xgb
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
# from sklearn.preprocessing import LabelEncoder
# from sklearn.linear_model import LinearRegression, ElasticNet
# from sklearn import linear_model

# # Feature Engineering
# inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/volume_output.csv'
# features = pd.read_csv(inputFile)

# # Merge hierarchy
# features = features.merge(r_hier, how='left', on='cost_object')

# features['unique_id'] = 'global' + '/' + features['region'] + '/' + features['cost_object'] + '/' + features['product']
# features = features[['unique_id', 'region', 'cost_object', 'product']].drop_duplicates()

# Y_train_df_ml = Y_train_df.merge(features, on='unique_id', how='left')

# # Creating LabelEncoder instance
# label_encoder = LabelEncoder()

# # Encoding 'cost_object' and 'product'
# Y_train_df_ml['region_encoded'] = label_encoder.fit_transform(Y_train_df_ml['region'])
# Y_train_df_ml['cost_object_encoded'] = label_encoder.fit_transform(Y_train_df_ml['cost_object'])
# Y_train_df_ml['product_encoded'] = label_encoder.fit_transform(Y_train_df_ml['product'])

# # Subset
# Y_train_df_ml2 = Y_train_df_ml[['unique_id', 'ds', 'y', 'region_encoded', 'cost_object_encoded','product_encoded']]

# models = [
#     # linear_model.BayesianRidge(),
#     # ElasticNet(alpha=0.08, l1_ratio=0.5),
#     lgb.LGBMRegressor(verbosity=-1),
#     xgb.XGBRegressor(),
#     RandomForestRegressor(random_state=0),
# ]

# from mlforecast import MLForecast
# from mlforecast.target_transforms import Differences
# from numba import njit
# from window_ops.expanding import expanding_mean
# from window_ops.rolling import rolling_mean


# @njit
# def rolling_mean_quarter(x):
#     return rolling_mean(x, window_size=6)


# fcst = MLForecast(
#     models=models,
#     freq='MS',
#     lags=[1, 2, 3, 6, 12],
#     lag_transforms={
#         1: [expanding_mean],
#         7: [rolling_mean_quarter]
#     },
#     date_features=['month']
#     # ,target_transforms=[Differences([1])],
# )

# fcst.fit(Y_train_df_ml2, static_features=['region_encoded', 'cost_object_encoded', 'product_encoded'])

# # Predict
# ml_models = fcst.predict(fct_periods)
# ml_models.head()

In [139]:
# Create data to plot
data2plot = Y_hier_df.reset_index(drop=True)
data2plot['ds'] = pd.to_datetime(data2plot['ds'])

# Create a new column 'Actuals'
fct_st_date = pd.to_datetime(fct_st_date)
data2plot['Actuals'] = np.where(data2plot['ds'] >= fct_st_date, data2plot['y'], np.nan)

# Replace 'y' values with NaN where the condition is met
data2plot['y'] = np.where(data2plot['ds'] >= fct_st_date, np.nan, data2plot['y'])
data2plot.rename(columns={'y': 'Actuals (Train)'}, inplace=True)

# Add budget, pam
data2plot = data2plot.merge(budget_hier_df[['unique_id', 'ds', 'budget']], on=['ds', 'unique_id'], how='left')
data2plot = data2plot.merge(pam_hier_df[pam_hier_df['ds']>=fct_st_date][['unique_id', 'ds', 'pam']], on=['ds', 'unique_id'], how='left')

# Add causal model
fcst_gc = fcst_gc[['unique_id', 'ds', 'AutoARIMA']]
fcst_gc['ds'] = pd.to_datetime(fcst_gc['ds'])
data2plot = data2plot.merge(fcst_gc[fcst_gc['ds']>=fct_st_date], on=['ds', 'unique_id'], how='left')

# Add hierarchical
Y_rec_df['ds'] = pd.to_datetime(Y_rec_df['ds'])
data2plot = data2plot.merge(Y_rec_df[Y_rec_df['ds']>=fct_st_date], on=['ds', 'unique_id'], how='left')

# Add tgpt
tgpt_cov['ds'] = pd.to_datetime(tgpt_cov['ds'])
data2plot = data2plot.merge(tgpt_cov[tgpt_cov['ds']>=fct_st_date][['unique_id', 'ds', 'TimeGPT']], on=['ds', 'unique_id'], how='left')

# Add hierarchical tgpt
Y_rec_df_tgpt['ds'] = pd.to_datetime(Y_rec_df_tgpt['ds'])
data2plot = data2plot.merge(Y_rec_df_tgpt[Y_rec_df_tgpt['ds']>=fct_st_date], on=['ds', 'unique_id'], how='left')

# # Add tgpt+covariate
# timegpt_fcst_ex_vars_df['ds'] = pd.to_datetime(timegpt_fcst_ex_vars_df['ds'])
# data2plot = data2plot.merge(timegpt_fcst_ex_vars_df[timegpt_fcst_ex_vars_df['ds']>=fct_st_date], on=['ds', 'unique_id'], how='left')

# # Add ml_models
# data2plot = data2plot.merge(ml_models[ml_models['ds']>=fct_st_date], on=['ds', 'unique_id'], how='left')

# # Add price
# Y_hier_df_p['ds'] = pd.to_datetime(Y_hier_df_p['ds'])
# data2plot = data2plot.merge(Y_hier_df_p[Y_hier_df_p['ds']<fct_st_date], on=['ds', 'unique_id'], how='left')
# Y_hier_df_p['ds'] = pd.to_datetime(Y_hier_df_p['ds'])
# data2plot = data2plot.merge(Y_hier_df_p[Y_hier_df_p['ds']>=fct_st_date], on=['ds', 'unique_id'], how='left')
# Y_hat_df_p['ds'] = pd.to_datetime(Y_hat_df_p['ds'])
# data2plot = data2plot.merge(Y_hat_df_p[Y_hat_df_p['ds']>=fct_st_date], on=['ds', 'unique_id'], how='left')

# # Rename columns
# data2plot.columns = ['unique_id', 'ds', 'Actuals (Train)', 'Actuals', 'Budget','PAM', 'AETS', 'AETS_H', 'TGPT', 'TGPT_H', 'TGPT_C', 'Prophet', 'LGBM', 'XGB', 'RF', 'Price (Train)', 'Price (Test)', 'Price (FCT)']

# Rename columns
data2plot.columns = ['unique_id', 'ds', 'Actuals (Train)', 'Actuals', 'Budget','PAM', 'AA', 'AETS', 'AETS_H', 'TGPT_C', 'TGPT', 'TGPT_H']

# Filter to end date
data2plot = data2plot[data2plot['ds']<=fct_end_date]

In [140]:
########################
# CHECK WINNER
########################
# Filter data2plot
data4metrics = data2plot[data2plot['ds'].isin(budget['ds'].unique())]
data4metrics = data4metrics[data4metrics['ds']<=fct_end_date]

numeric_cols = data4metrics.columns.drop(['unique_id', 'ds'])

# Sum up the values for each unique_id
summed_df = data4metrics.groupby('unique_id')[numeric_cols].sum()

# Remove columns with no actuals
summed_df = summed_df[summed_df['Actuals'] != 0]

# Calculate percentage differences from 'Actuals'
percentage_diff = summed_df.subtract(summed_df['Actuals'], axis=0).div(summed_df['Actuals'], axis=0).abs()

# Drop the 'Actuals' column as we don't need to compare it with itself
# percentage_diff.drop(columns=['Actuals', 'Actuals (Train)', 'Prophet', 'XGB', 'TGPT_H', 'TGPT_C', 'AETS_H', 'TGPT', 'LGBM', 'RF', 'PAM'], inplace=True)
percentage_diff.drop(columns=['Actuals', 'Actuals (Train)', 'PAM', 'AA', 'TGPT', 'TGPT_H', 'TGPT_C'], inplace=True)

# Find the column with the lowest percentage difference for each unique_id
min_diff_col = percentage_diff.idxmin(axis=1)

# Add this as a new column in your original DataFrame
data4metrics['lowest_diff_col'] = data4metrics['unique_id'].map(min_diff_col)

winner = data4metrics[data4metrics['unique_id'].isin(budget_hier_df['unique_id'])].groupby('lowest_diff_col')

winner['unique_id'].nunique()

lowest_diff_col
AETS      106
AETS_H    103
Budget    216
Name: unique_id, dtype: int64

In [141]:
bud_winners = winner.get_group('Budget')['unique_id'].unique()
ts2fix = data2plot[data2plot['unique_id'].isin(bud_winners)]

In [142]:
########################
# PLOT
########################
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import pandas as pd

# Update the function to include filtering based on 'unique_id'
def plot_data(unique_id, x_column, y_columns):
    # Filter data based on selected unique_id
    filtered_data = data2plot[data2plot['unique_id'] == unique_id]

    # Plotting
    plt.figure(figsize=(8, 5))
    for y_column in y_columns:
        plt.plot(filtered_data[x_column], filtered_data[y_column], label=y_column)
    plt.xlabel(x_column)
    plt.ylabel('Values')
    plt.title(f'Multiple Y-Axis Plot for unique_id {unique_id}')
    plt.legend()
    plt.show()

    # Displaying the sum table
    display_data = filtered_data[[x_column] + list(y_columns)].copy()

    # Create a sum row
    sum_values = {x_column: 'Sum'}
    for col in list(y_columns):
        sum_values[col] = display_data[col].sum()
    sum_row = pd.DataFrame([sum_values])

    display_data = pd.concat([display_data, sum_row], ignore_index=True)

    # Calculating and displaying the percentage difference table
    perc_diff_rows = []
    actuals_sum = sum_values['Actuals']
    for y_col in y_columns:
        y_sum = sum_values[y_col]
        perc_diff = ((y_sum - actuals_sum) / actuals_sum) * 100 if actuals_sum != 0 else None
        perc_diff_rows.append({'Y Column': y_col, 'Percentage Difference': perc_diff})

    perc_diff_data = pd.DataFrame(perc_diff_rows)
    display(perc_diff_data)
    display(display_data)

data2use = ts2fix
# data2use = data2plot

# Create widgets
unique_id_selector = widgets.SelectionSlider(
    options=data2use['unique_id'].unique(),
    description='unique_id:',
    orientation='horizontal',
    readout=True
)

x_column = widgets.Dropdown(options=data2use.columns, value=data2use.columns[1], description='X-axis:')
y_columns = widgets.SelectMultiple(options=data2use.columns, value=[data2use.columns[2], data2use.columns[3], data2use.columns[4], data2use.columns[5], data2use.columns[6], data2use.columns[7], data2use.columns[8]], description='Y-axis:')

# Display interactive plot
interact(plot_data, unique_id=unique_id_selector, x_column=x_column, y_columns=y_columns)

interactive(children=(SelectionSlider(description='unique_id:', options=('global', 'global/CIS', 'global/East …

<function __main__.plot_data(unique_id, x_column, y_columns)>

In [131]:
bud_winners

array(['global', 'global/CIS', 'global/East Europe',
       'global/Greater China', 'global/Japan', 'global/SESA',
       'global/CIS/D_I_CIS_BEL - Belarus',
       'global/CIS/D_I_CIS_RUS - Russia',
       'global/East Europe/D_E_HU - Hungary',
       'global/East Europe/D_E_PO - Poland',
       'global/East Europe/D_E_UA - Ukraine',
       'global/East Europe/D_GR10_CY - Cyprus',
       'global/East Europe/D_GR10_GR - Greece',
       'global/Greater China/D_CN_TOTAL - China Total',
       'global/Greater China/D_HK_TOTAL - Hong Kong Total',
       'global/Greater China/D_TW_TOTAL - Taiwan Total',
       'global/Japan/JP10 - Astellas Pharma Inc',
       'global/LATAM/D_I_AR - Domestic Argentina',
       'global/LATAM/D_I_CO - Domestic Colombia',
       'global/LATAM/D_I_LATAM_REST_OTH - Domestic Rest of Latam Others',
       'global/LATAM/D_I_MX - Domestic Mexico',
       'global/LATAM/D_I_MX_DB - Domestic Mexico Distributor',
       'global/MEA/D_E_IL - Israel', 'global/MEA/D_I_AE - 